In [46]:
import os
import numpy as np
import itertools
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 20)

<br>

# Functions

In [2]:
def M2Q(df_in):
    d = {"M03":"Q1", "M06":"Q2", "M09":"Q3", "M12":"Q4"}
    df_out = df_in[df_in['TIME'].apply(lambda x: x[-3:] in list(d.keys()))].copy()
    df_out.TIME = df_out['TIME'].apply(lambda x: x[:4]+d[x[-3:]]).copy()
    return(df_out)

def Qp1(df_in):
    d = {"Q1":"Q2", "Q2":"Q3", "Q3":"Q4", "Q4":"Q1"}
    df_out = df_in.copy()
    df_out.TIME = df_out['TIME'].apply(lambda x: str(int(x[:-2])+1)+d[x[-2:]] if x[-1:] == "4" else x[:-2]+d[x[-2:]]).copy()
    return(df_out)

<br>

# Data

In [3]:
path = "./data/csvs/"
!sed -i "s/://g" ./data/*

In [4]:
#!head -n 1 data/sts_trtu_q_Label.csv

>## Quarterly

In [5]:
# DATASET Construction - quarterly data - index (2015 = 100) (NACE Rev. 2) [ei_isbu_q]
ei_isbu_q = pd.read_csv("./data/ei_isbu_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
ei_isbu_q_lag = Qp1(ei_isbu_q.copy())

# DATASET Final consumption aggregates [namq_10_fcs]
namq_10_fcs = pd.read_csv("./data/namq_10_fcs_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
namq_10_fcs_lag = Qp1(namq_10_fcs.copy())

# DATASET: GDP and main components  (output, expenditure and income) [namq_10_gdp]
namq_10_gdp = pd.read_csv("./data/namq_10_gdp_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
namq_10_gdp_lag = Qp1(namq_10_gdp.copy())

# DATASET Building permits - quarterly data [sts_cobp_q]
sts_cobp_q = pd.read_csv("./data/sts_cobp_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
sts_cobp_q_lag = Qp1(sts_cobp_q.copy())

# DATASET Production in construction - quarterly data [sts_copr_q]
sts_copr_q = pd.read_csv("./data/sts_copr_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
sts_copr_q_lag = Qp1(sts_copr_q.copy())

# DATASET Production in industry - quarterly data [sts_inpr_q]
sts_inpr_q = pd.read_csv("./data/sts_inpr_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
sts_inpr_q_lag = Qp1(sts_inpr_q.copy())

# DATASET Turnover in industry, total - quarterly data [sts_intv_q]
sts_intv_q = pd.read_csv("./data/sts_intv_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
sts_intv_q_lag = Qp1(sts_intv_q.copy())

# DATASET Turnover and volume of sales in wholesale and retail trade - quarterly data [sts_trtu_q]
sts_trtu_q = pd.read_csv("./data/sts_trtu_q_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna()
sts_trtu_q_lag = Qp1(sts_trtu_q.copy())

>## Monthly

In [6]:
# DATASET Euro-zone Business Climate Indicator - monthly data [ei_bsci_m_r2]
ei_bsci_m_r2_1 = M2Q(pd.read_csv("./data/ei_bsci_m_r2_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna())

# DATASET Sentiment indicators - monthly data [ei_bssi_m_r2]
ei_bssi_m_r2 = M2Q(pd.read_csv("./data/ei_bssi_m_r2_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna())

# DATASET Effective exchange rates indices - monthly data [ei_mfef_m]
ei_mfef_m = M2Q(pd.read_csv("./data/ei_mfef_m_1_Data.csv", low_memory=False, usecols=lambda x: x not in ["Flag and Footnotes"]).dropna())

**Dealing with ids:**

In [7]:
for df in [ei_isbu_q, ei_isbu_q_lag, namq_10_fcs, namq_10_fcs_lag, namq_10_gdp, 
           namq_10_gdp_lag, sts_cobp_q, sts_cobp_q_lag, sts_copr_q, sts_copr_q_lag, 
           sts_inpr_q, sts_inpr_q_lag, sts_intv_q, sts_intv_q_lag, sts_trtu_q, 
           sts_trtu_q_lag, ei_isbu_q, ei_isbu_q_lag, namq_10_fcs, namq_10_fcs_lag, 
           namq_10_gdp, namq_10_gdp_lag, sts_cobp_q, sts_cobp_q_lag, sts_copr_q, sts_copr_q_lag, 
           sts_inpr_q, sts_inpr_q_lag, sts_intv_q, sts_intv_q_lag, sts_trtu_q, sts_trtu_q_lag, ei_bsci_m_r2_1, ei_bssi_m_r2, ei_mfef_m]:
    df["ID"] = df["TIME"]+df["GEO"]
    df.Value = df.Value.replace(regex=r'[,]+', value='').astype(np.float)
    


># OECD

In [8]:
def OECD():
    GEO_LABEL2GEO = dict(zip(namq_10_gdp.GEO_LABEL.values, namq_10_gdp.GEO.values)); GEO_LABEL2GEO["Germany"] = "DE"
    df_in = pd.read_csv("data/KEI_13112018143436794_[v1-oecd].csv")
    df_in["GEO"] = df_in.Country.map(GEO_LABEL2GEO)
    df_in = df_in[df_in.GEO.notnull()]
    df_in["ID"] = df_in.TIME.str[:4]+df_in.TIME.str[-2:]+df_in.GEO
    return(df_in)

In [9]:
OECD = OECD()

<br>

# $C$

>## $Cd$

In [55]:
df_in = pd.DataFrame(sorted(list(set(namq_10_fcs.ID))), columns=["ID"])

###########################################################################################

# Chain linked volumes, percentage change compared to same period in previous year
var = "CLV_PCH_SM"
selec = namq_10_fcs[(namq_10_fcs.NA_ITEM=="P311_S14")&(namq_10_fcs.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

## Chain linked volumes, percentage change on previous period
#var = "CLV_PCH_PRE"
#selec = namq_10_fcs_lag[(namq_10_fcs_lag.NA_ITEM=="P311_S14")&(namq_10_fcs_lag.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
#df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Business Climate Indicator
var = "BS-BCI"
dic_BCI = dict(zip(ei_bsci_m_r2_1.TIME.values, ei_bsci_m_r2_1.Value.values))
df_in[var] = df_in.ID.str[:-2].map(dic_BCI)

# Retail Confidence Indicator
var = "BS-RCI-BAL"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Services Confidence Indicator
var = "BS-SCI-BAL"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Consumer Confidence Indicator
var = "BS-CSMCI-BAL"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Production in industry: MIG - Durable Consumer Goods
var = "MIG_DCOG"
selec = sts_inpr_q[sts_inpr_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Manufacture of motor vehicles, trailers, semi-trailers and of other transport equipment
var = "C29_C30"
selec = sts_intv_q[sts_intv_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Harmonised unemployment rate all persons, s.a.
var = "LRHUTTTT"
selec = OECD[OECD.SUBJECT==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Passenger car registrations, s.a.
var = "SLRTCR03"
selec = OECD[OECD.SUBJECT==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

###########################################################################################

# Dummies
countries = set(namq_10_fcs.GEO); countries.remove('PT')
for country in sorted(list(countries)):
    df_in[country] = (df_in.ID.str[-2:] == country).astype(int)

Cd = df_in.dropna().copy(); Cd.set_index("ID", inplace=True)
Cd = Cd.loc[:, (Cd != 0).any(axis=0)].copy()
del var, selec, df_in

###########################################################################################

# Save
Cd.to_csv("./Cd.csv")

###########################################################################################

# Regression
X = sm.add_constant(Cd.loc[:, Cd.columns != "CLV_PCH_SM"].copy())
y = Cd.loc[:, "CLV_PCH_SM"].copy()
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             CLV_PCH_SM   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     46.70
Date:                Wed, 14 Nov 2018   Prob (F-statistic):          7.97e-148
Time:                        03:30:48   Log-Likelihood:                -3777.8
No. Observations:                1092   AIC:                             7606.
Df Residuals:                    1067   BIC:                             7731.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -4.9055      3.843     -1.277   

>## $Cnd$

In [54]:
df_in = pd.DataFrame(sorted(list(set(namq_10_fcs.ID))), columns=["ID"])

###########################################################################################

# Chain linked volumes, percentage change compared to same period in previous year
var = "CLV_PCH_SM"
selec = namq_10_fcs[(namq_10_fcs.NA_ITEM=="P312N_S14")&(namq_10_fcs.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Chain linked volumes, percentage change on previous period
#var = "CLV_PCH_PRE"
#selec = namq_10_fcs_lag[(namq_10_fcs_lag.NA_ITEM=="P312N_S14")&(namq_10_fcs_lag.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
#df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Business Climate Indicator
var = "BS-BCI"
dic_BCI = dict(zip(ei_bsci_m_r2_1.TIME.values, ei_bsci_m_r2_1.Value.values))
df_in[var] = df_in.ID.str[:-2].map(dic_BCI)

# Retail Confidence Indicator
var = "BS-RCI-BAL"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Services Confidence Indicator
var = "BS-SCI-BAL"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Consumer Confidence Indicator
var = "BS-CSMCI-BAL"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Production in industry: MIG - Non-Durable Consumer Goods
var = "MIG_NDCOG"
selec = sts_inpr_q[sts_inpr_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Retail sale of food, beverages and tobacco
var = "G47_FOOD"
selec = sts_trtu_q[sts_trtu_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Retail sale of non-food products (except fuel)
var = "G47_NFOOD_X_G473"
selec = sts_trtu_q[sts_trtu_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Retail trade, except of motor vehicles, motorcyles and fuel
var = "G47_X_G473"
selec = sts_trtu_q[sts_trtu_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Harmonised unemployment rate all persons, s.a.
var = "LRHUTTTT"
selec = OECD[OECD.SUBJECT==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

###########################################################################################

# Dummies
countries = set(namq_10_fcs.GEO); countries.remove('PT')
for country in sorted(list(countries)):
    df_in[country] = (df_in.ID.str[-2:] == country).astype(int)

Cnd = df_in.dropna().copy().set_index("ID")
Cnd = Cnd.loc[:, (Cnd != 0).any(axis=0)].copy()

del var, selec, df_in

###########################################################################################

# Save
Cnd.to_csv("./Cnd.csv")

###########################################################################################

# Regression
X = sm.add_constant(Cnd.loc[:, Cnd.columns != "CLV_PCH_SM"].copy())
y = Cnd.loc[:, "CLV_PCH_SM"].copy()
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             CLV_PCH_SM   R-squared:                       0.521
Model:                            OLS   Adj. R-squared:                  0.520
Method:                 Least Squares   F-statistic:                     412.8
Date:                Wed, 14 Nov 2018   Prob (F-statistic):               0.00
Time:                        03:29:51   Log-Likelihood:                -21825.
No. Observations:                9889   AIC:                         4.370e+04
Df Residuals:                    9862   BIC:                         4.390e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                8.3155      0.398  

<br>

# $I$

>## $Igfcf$

In [39]:
df_in = pd.DataFrame(sorted(list(set(namq_10_gdp.ID))), columns=["ID"])

###########################################################################################

# Chain linked volumes, percentage change compared to same period in previous year
var = "CLV_PCH_SM"
selec = namq_10_gdp[(namq_10_gdp.NA_ITEM=="P51G")&(namq_10_gdp.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

## Chain linked volumes, percentage change on previous period
#var = "CLV_PCH_PRE"
#selec = namq_10_gdp_lag[(namq_10_gdp_lag.NA_ITEM=="P51G")&(namq_10_gdp_lag.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
#df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Business Climate Indicator
var = "BS-BCI"
dic_BCI = dict(zip(ei_bsci_m_r2_1.TIME.values, ei_bsci_m_r2_1.Value.values))
df_in[var] = df_in.ID.str[:-2].map(dic_BCI)

# Industrial Confidence Indicator
var = "BS-ICI-BAL"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

## Building permits index - New residential buildings
#var = "IS-PEI"
#selec = ei_isbu_q[ei_isbu_q.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
#df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Residential buildings, except residences for communities
var = "F_CC11_X_CC113"
selec = sts_cobp_q[sts_cobp_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Two- and more dwelling buildings
var = "F_CC112"
selec = sts_cobp_q[sts_cobp_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Construction confidence indicator
var = "BS-CCI-BAL"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Volume index of production
var = "PROD"
selec = sts_copr_q[sts_copr_q.INDIC_BT==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Production in industry: MIG - energy (except Section E)
var = "MIG_NRG_X_E"
selec = sts_inpr_q[sts_inpr_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Production in industry: MIG - capital goods
var = "MIG_CAG"
selec = sts_inpr_q[sts_inpr_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Production in industry: Mining and quarrying; manufacturing
var = "B_C"
selec = sts_inpr_q[sts_inpr_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Production in industry: Mining and quarrying; manufacturing (except MIG energy)
var = "B_C_X_MIG_NRG"
selec = sts_inpr_q[sts_inpr_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Production in industry: Manufacturing
var = "C"
selec = sts_inpr_q[sts_inpr_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", "C_prod"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Production in industry: Electricity, gas, steam and air conditioning supply
var = "D"
selec = sts_inpr_q[sts_inpr_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Manufacture of motor vehicles, trailers, semi-trailers and of other transport equipment
var = "C29_C30"
selec = sts_intv_q[sts_intv_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Manufacturing
var = "C"
selec = sts_intv_q[sts_intv_q.NACE_R2==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

###########################################################################################

# Dummies
countries = set(namq_10_gdp.GEO); countries.remove('PT')
for country in sorted(list(countries)):
    df_in[country] = (df_in.ID.str[-2:] == country).astype(int)

Igfcf = df_in.dropna().copy().set_index("ID")
Igfcf = Igfcf.loc[:, (Igfcf != 0).any(axis=0)].copy()
del var, selec, df_in

###########################################################################################

# Save
Igfcf.to_csv("./Igfcf.csv")

###########################################################################################

# Regression
X = sm.add_constant(Igfcf.loc[:, Igfcf.columns != "CLV_PCH_SM"].copy())
y = Igfcf.loc[:, "CLV_PCH_SM"].copy()
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             CLV_PCH_SM   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.480
Method:                 Least Squares   F-statistic:                     38.87
Date:                Wed, 14 Nov 2018   Prob (F-statistic):          5.78e-159
Time:                        02:15:50   Log-Likelihood:                -4387.2
No. Observations:                1273   AIC:                             8838.
Df Residuals:                    1241   BIC:                             9003.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             23.9768      3.413      7.

<br>

# $G$

<br>

# $X$

In [53]:
df_in = pd.DataFrame(sorted(list(set(namq_10_gdp.ID))), columns=["ID"])

###########################################################################################

# Chain linked volumes, percentage change compared to same period in previous year
var = "CLV_PCH_SM"
selec = namq_10_gdp[(namq_10_gdp.NA_ITEM=="P51G")&(namq_10_gdp.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Chain linked volumes, percentage change on previous period
var = "CLV_PCH_PRE"
selec = namq_10_gdp_lag[(namq_10_gdp_lag.NA_ITEM=="P51G")&(namq_10_gdp_lag.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Economic sentiment indicator
var = "BS-ESI-I"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Consumer confidence indicator
var = "BS-CSMCI-BAL"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Retail confidence indicator
var = "BS-RCI-BAL"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")


###########################################




###########################################################################################

# Dummies
countries = set(namq_10_gdp.GEO); countries.remove('PT')
for country in sorted(list(countries)):
    df_in[country] = (df_in.ID.str[-2:] == country).astype(int)

Igfcf = df_in.copy().set_index("ID")
Igfcf = Igfcf.loc[:, (Igfcf != 0).any(axis=0)].copy()
del var, selec, df_in

###########################################################################################

# Save
Igfcf.to_csv("./Igfcf.csv")

###########################################################################################

# Regression
X = sm.add_constant(Igfcf.loc[:, Igfcf.columns != "CLV_PCH_SM"].copy())
y = Igfcf.loc[:, "CLV_PCH_SM"].copy()
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             CLV_PCH_SM   R-squared:                       0.281
Model:                            OLS   Adj. R-squared:                  0.271
Method:                 Least Squares   F-statistic:                     27.17
Date:                Wed, 14 Nov 2018   Prob (F-statistic):          3.11e-118
Time:                        03:28:58   Log-Likelihood:                -7485.2
No. Observations:                1974   AIC:                         1.503e+04
Df Residuals:                    1945   BIC:                         1.519e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -43.8957      5.750     -7.635   

<br>

# $M$

In [52]:
df_in = pd.DataFrame(sorted(list(set(namq_10_gdp.ID))), columns=["ID"])

###########################################################################################

# Chain linked volumes, percentage change compared to same period in previous year
var = "CLV_PCH_SM"
selec = namq_10_gdp[(namq_10_gdp.NA_ITEM=="P7")&(namq_10_gdp.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", "M"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Cd
selec = namq_10_fcs[(namq_10_fcs.NA_ITEM=="P311_S14")&(namq_10_fcs.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", "Cd"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Cnd
selec = namq_10_fcs[(namq_10_fcs.NA_ITEM=="P312N_S14")&(namq_10_fcs.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", "Cnd"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# G
selec = namq_10_gdp[(namq_10_gdp.NA_ITEM=="P3_S13")&(namq_10_gdp.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", "G"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Igfcf
selec = namq_10_gdp[(namq_10_gdp.NA_ITEM=="P51G")&(namq_10_gdp.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", "Igfcf"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Economic sentiment indicator
var = "BS-ESI-I"
selec = ei_bssi_m_r2[ei_bssi_m_r2.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Real Effective Exchange Rate (deflator consumer price indices - 42 trading partners)
var = "REER42CPI"
selec = ei_mfef_m[ei_mfef_m.INDIC==var][["ID", "Value"]]; selec.columns = ["ID", var]
df_in = pd.merge(df_in, selec, on='ID', how="inner")


#############################################

# Chain linked volumes, percentage change compared to same period in previous year
var = "CLV_PCH_SM"
selec = namq_10_gdp_lag[(namq_10_gdp_lag.NA_ITEM=="P7")&(namq_10_gdp.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", "M"+"_qoq"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Cd
selec = namq_10_fcs_lag[(namq_10_fcs_lag.NA_ITEM=="P311_S14")&(namq_10_fcs.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", "Cd"+"_qoq"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Cnd
selec = namq_10_fcs_lag[(namq_10_fcs_lag.NA_ITEM=="P312N_S14")&(namq_10_fcs.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", "Cnd"+"_qoq"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# G
selec = namq_10_gdp_lag[(namq_10_gdp_lag.NA_ITEM=="P3_S13")&(namq_10_gdp.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", "G"+"_qoq"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

# Igfcf
selec = namq_10_gdp_lag[(namq_10_gdp_lag.NA_ITEM=="P51G")&(namq_10_gdp.UNIT==var)][["ID", "Value"]]; selec.columns = ["ID", "Igfcf"+"_qoq"]
df_in = pd.merge(df_in, selec, on='ID', how="inner")

###########################################################################################

# Dummies
countries = set(namq_10_gdp.GEO); countries.remove('PT')
for country in sorted(list(countries)):
    df_in[country] = (df_in.ID.str[-2:] == country).astype(int)

M = df_in.copy().set_index("ID")
M = M.loc[:, (M != 0).any(axis=0)].copy()
del var, selec, df_in

###########################################################################################

# Save
M.to_csv("./M.csv")

###########################################################################################

# Regression
X = sm.add_constant(M.loc[:, M.columns != "M"].copy())
y = M.loc[:, "M"].copy()
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      M   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.732
Method:                 Least Squares   F-statistic:                     145.9
Date:                Wed, 14 Nov 2018   Prob (F-statistic):               0.00
Time:                        03:28:17   Log-Likelihood:                -4934.1
No. Observations:                1701   AIC:                             9934.
Df Residuals:                    1668   BIC:                         1.011e+04
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -12.3136      2.343     -5.255      0.0

<br>

<br>

# Prepare to $\textbf{R}$

In [103]:
# namq_10_gdp
selec = namq_10_gdp[(namq_10_gdp.GEO == "PT") & (namq_10_gdp.UNIT == "CLV10_MEUR")]
for item in list(set(selec.NA_ITEM)):
    selec[selec.NA_ITEM==item][["TIME", "Value"]].to_csv("R/"+item+".csv", index=False)
##

# namq_10_fcs